In [1]:
import pandas as pd
import scipy.constants
from scipy.optimize import curve_fit
from scipy.integrate import quad
from sklearn.metrics import r2_score
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib.transforms as transforms
from matplotlib.ticker import LogFormatter
import os
import fnmatch
%matplotlib qt
# %matplotlib inline
import datetime as dt
pd.set_option('display.max_rows', 100) 

## Reading in the data and preparing DataFrames

#### MID

In [2]:
#read in the file and create dataframe
#excplicitly state the part
#path = r'C:\Users\einot\OneDrive\Documents\CERN VScode\CERN-Python\4K desorption measurements\Cu sample\EGA fully in\S1 no preinjection at cold temps'
#mid = pd.read_csv(os.path.join(path, "MID.tsv"), sep="\t", skiprows=lambda x: x<=11, on_bad_lines="skip")

#List molecule masses used in MID scan

#masslist = [2,4,12,15,16,18,28,32,40,44]

#Define datetime format
#MID_datetime = "%Y/%m/%d %H:%M:%S.%f"

#Reformat time values
def timeformat(df,format):
    new_df = df.copy()
    for idx, col in enumerate(df):
        if idx % 2 == 0:  # Check if the column index is even (every other column)
            pandas_timestamp = pd.to_datetime(df[col], format=format)
            timestamp = pandas_timestamp.dt.strftime('%d-%m-%Y %H:%M:%S')
            new_df[col] = timestamp  # Replace the time column in the new dataframe
    return new_df

#call the function with correct parameters to modify MID dataframe
#mid = timeformat(mid,MID_datetime)

#### Vaclogger

In [3]:
""" #Reading in the file
vaclog = pd.read_csv(os.path.join(path, "vaclog"), sep="\t")

#Define original datetime format
vaclog_datetime = "%d/%m/%Y %H:%M:%S"
 """
#Adding an elapsed time column for temperature fits
def elapsed_time(df,timecol,format):
    new_df = df.copy()
    pandas_timestamp = pd.to_datetime(new_df[timecol],format=format)
    runtime = (pandas_timestamp-pandas_timestamp[0]).dt.total_seconds()
    insert_idx = df.columns.get_loc(timecol) + 1  # Get the index to insert the new column
    new_df.insert(insert_idx,"Elapsed time", runtime)
    reformat = pandas_timestamp.dt.strftime('%d-%m-%Y %H:%M:%S') #New timestamp format for plotting
    new_df[timecol] = reformat
    new_df[timecol] = pd.to_datetime(new_df[timecol],format='%d-%m-%Y %H:%M:%S') #convert to pd datetime
    
    return new_df
""" 
#call the function with correct parameters to modify vaclogger dataframe
vaclog = elapsed_time(vaclog,"Time",vaclog_datetime) """

' \n#call the function with correct parameters to modify vaclogger dataframe\nvaclog = elapsed_time(vaclog,"Time",vaclog_datetime) '

In [4]:
def process_data(root):
    #Define datetime formats
    MID_datetime = "%Y/%m/%d %H:%M:%S.%f"
    vaclog_datetime = "%d/%m/%Y %H:%M:%S"
    hv_datetime = "%d/%m/%Y %H:%M:%S.%f"
    
    #Create an empty dictionary to store processed dataframes
    dataframes = {}

    # Iterate over the subdirectories starting from the specified directory
    for dirpath, dirs, files in os.walk(root):
        for filename in files:
            filepath = os.path.join(dirpath,filename)
            #print(filepath)
            # Create a variable name using the relative path
            relative_path = os.path.relpath(root, dirpath)
            #print(relative_path)
            variable_name = os.path.join(relative_path, os.path.splitext(filename)[0])
            #print(variable_name)
            #Read in the MID file
            if fnmatch.fnmatch(filename,"*MID*.tsv"):
                mid = pd.read_csv(filepath, sep="\t", skiprows=lambda x: x<=11, on_bad_lines="skip")
                mid = timeformat(mid, MID_datetime)

                # Create a variable name using the base name of the MID filename
                #variable_name = os.path.splitext(filename)[0]

                # Store the MID dataframe using the variable name
                dataframes[variable_name] = mid

            #Read in the vaclog file
            elif fnmatch.fnmatch(filename,"*vaclog*"):
                vaclog = elapsed_time(pd.read_csv(filepath, sep="\t"), "Time", vaclog_datetime)
                
                # Create a variable name using the base name of the MID filename
                #variable_name = os.path.splitext(filename)[0]

                # Store the vaclog dataframe using the variable name
                dataframes[variable_name] = vaclog

            #Read in the hivolta file
            elif fnmatch.fnmatch(filename,"*hv log*"):
                hv = pd.read_csv(filepath, sep=",")
                hv["hv_grid"] = [abs(element) * 1e-6 for element in hv["IMon1"]]
                hv["I_em"] = [abs(element) * 1e-6 for element in hv["IMon2"]]
                hv["datetime"] = hv['Date'] + ' ' + hv['Time']
                hv = elapsed_time(hv, "datetime", hv_datetime)

                # Create a variable name using the base name of the MID filename
                #variable_name = os.path.splitext(filename)[0]
                
                # Store the hv dataframe using the variable name
                dataframes[variable_name] = hv   
            
    return dataframes




## Data processing

In [5]:
#Accessing the files
root = os.getcwd()
data = process_data(root)
#print(vaclog)
# Access the processed dataframes using the variable names
for variable_name, dataframe in data.items():
    print(variable_name)
    print(dataframe)
    print("-----------------")

#%xl_set data --cell Sheet1!A2


.\vaclog
               Live comments                Time  Elapsed time  \
0      starting lhe transfer 2022-12-08 11:07:05           0.0   
1                        NaN 2022-12-08 11:07:12           7.0   
2                        NaN 2022-12-08 11:07:20          15.0   
3                        NaN 2022-12-08 11:07:27          22.0   
4                        NaN 2022-12-08 11:07:34          29.0   
...                      ...                 ...           ...   
11420                    NaN 2022-12-09 09:55:01       82076.0   
11421                    NaN 2022-12-09 09:55:08       82083.0   
11422                    NaN 2022-12-09 09:55:15       82090.0   
11423                    NaN 2022-12-09 09:55:23       82098.0   
11424                    NaN 2022-12-09 09:55:30       82105.0   

       injection 100mbar      Barion_2      Barion_1  DUAL experiment  \
0                 78.515  4.770000e-09  1.430000e-09     4.996000e-09   
1                 78.515  4.770000e-09  1.430000e-09

#### CernOx R-T conversion 

In [6]:
def tempconvert(df):
    new_df = df.copy()
    #Temperature curve for CERNOX - for temp stability
    A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]

    #specify fit parameters A, data (Resistance values)
    def polyfit(param,data):
        total=[]
        for j in data: 
            exp=0
            for i in range(len(param)):
                exp += (param[i]/(math.log10(j))**i)
            total.append(10**exp)
        return(total)
    col_loc = int(df.columns.get_loc("T-CERNOX") + 1)    
    new_df.insert(col_loc, "CernOx Temp", polyfit(A, new_df["T-CERNOX"]))
    return new_df


#### LHe level

In [7]:
def linfit(x, a, b):
        return a * x + b
def lhe_fit(y_val, x_val): #Normal fit without extended x-axis
    # Curve fit for the data
    params, cov = curve_fit(linfit, x_val, y_val)

    # Straight line parameters
    a, b = params

    # Calculate the number of data points needed to reach b/a condition
    num_points = int(abs(b / a) / (x_val[1] - x_val[0])) + 1

    # Extend the fitted line to y_fit = 0
    x_fit = np.linspace(x_val[0], (-b/a), num_points)

    y_fit = linfit(x_fit, a, b)

    return x_fit, y_fit, a, b

def ext_fit(y_val, x_val, b_val): #extended fit with longer x-axis, user specified b-parameter
    # Curve fit for the data
    params, cov = curve_fit(linfit, x_val, y_val)

    # Straight line parameters
    a, b = params

    # Calculate the number of data points needed to reach b/a condition
    num_points = int(abs(b_val / a) / (x_val[1] - x_val[0])) + 1

    # Extend the fitted line to y_fit = 0
    x_fit_extended = np.linspace(x_val[0], (-b_val/a), num_points)

    y_fit_extended = linfit(x_fit_extended, a, b_val)

    return x_fit_extended, y_fit_extended, a, b


#### Coefficients for gauge readings
Applied for the gauge readings using the formula below:
    $$
    \frac{p_2}{p_1}=\sqrt \frac{T_2}{T_1}
    $$
Here $p_2$ and $T_2$ are the pressure and temperature in the cold part and $p_1$, $T_1$ are the pressure, temperature the gauge is exposed to.

In [8]:
#Thermal transpiration
T2 = 4.2
T1 = 293
tt_coef = np.sqrt(T2/T1)
print("thermal transpiration coef:",tt_coef)
#N2 to H2 equivalent conversion
CF_h2 = 0.38
def gauge_correction(df):
    new_df = df.copy()
    #apply correction factors
    ba1_h2_tt = df["Barion_1"]*tt_coef/CF_h2
    ba2_h2_tt = df["Barion_2"]*tt_coef/CF_h2
    ba1_h2 = df["Barion_1"]*CF_h2
    ba2_h2 = df["Barion_2"]*CF_h2
    #print(ba1[0])
    #print(df["Barion_1"][0])
    loc1 = int(df.columns.get_loc("Barion_1") + 1)
    loc2 = int(df.columns.get_loc("Barion_1") + 2)
    loc3 = int(df.columns.get_loc("Barion_1") + 3)
    loc4 = int(df.columns.get_loc("Barion_1") + 4)
    new_df.insert(loc1,"Barion_1 H2 equiv",ba1_h2)
    new_df.insert(loc2,"Barion_2 H2 equiv",ba2_h2)
    new_df.insert(loc3,"Barion_1 H2+tt",ba1_h2_tt)
    new_df.insert(loc4,"Barion_2 H2+tt",ba2_h2_tt)

   # print(new_df.head)
    return new_df

thermal transpiration coef: 0.11972665112564158


#### Theoretical isotherms

In [9]:
#reading in other data
p_0 = 4.25e-5 #(Pa)
p_0_mbar=p_0/100
p_0_mbar
S_m = 2.39e15 #molecules/cm2
d_wallen = 3.61e4/scipy.constants.e**2 # ev^-2 to J^-2
def read_data(filepath,delim):
    df=pd.read_csv(filepath, delimiter=delim)#for wallen, use "\t"
    return df

wallen = read_data(r'C:\Users\etiirine\cernbox\Documents\etiirinen\Python\H2 Adsorption isotherms\H2 adsorption isotherm on HiLumi sample\Wallen.csv',"\t")


In [10]:
#constants
b_hobson=1./528**2
print(b_hobson)
gas_constant=1.9865
d_CWB = 3075/scipy.constants.e**2 # J^-2
#Hobson and DRK isotherms
def theta_hobson(p):
    return S_m*np.exp(-b_hobson*(gas_constant*4.2*np.log(p/p_0))**2)
def theta_drk(p,d_const):
    return S_m*np.exp(-d_const*(scipy.constants.k*4.2*np.log(p/p_0))**2)
fit_values = np.logspace(-18,-4,50) #Pa
thetas=np.logspace(-7,0,100)

3.5870064279155188e-06


In [11]:
""" #Define pressure on sample
#find local minima:
def theoretic_sample_p(df):
    new_df = df.copy()
    #Create a variable for H2 equiv thermal transpiration corrected Barion 2 values
    ba2 = df["Barion_2 corr"].values
    minima_idx = scipy.signal.argrelmin(ba2,order=10)
    #Find the pressure values corresponding to the indexes found
    minima_pressure = ba2[minima_idx]
    #Find the elapsed time values
    elapsed_time_minima = df["Elapsed time"].values[minima_idx]
    # Vectorize the theta_hobson function to work with NumPy arrays
    theta_hobson_vec = np.vectorize(theta_hobson)
    # Perform curve fitting to get the parameters
    popt, pcov = curve_fit(theta_hobson_vec, minima_pressure, elapsed_time_minima, p0=1)

    # Calculate the fitted elapsed time values using the theta_hobson function
    fitted_elapsed_time = theta_hobson_vec(minima_pressure, *popt)

    #Create a polynomial fit that goes through these points
    # Add data to new_df
    col_loc = int(df.columns.get_loc("Barion_2 corr") + 1)
    new_df.insert(col_loc, "sample pressure", np.nan)
    new_df.loc[minima_idx[0], "sample pressure"] = minima_pressure
    new_df["sample pressure"] = fitted_elapsed_time
    #add data to new_df
    #col_loc = int(df.columns.get_loc("Barion_2 corr") + 1)    
    #new_df.insert(col_loc,"sample pressure",minima)
    return new_df

 """

' #Define pressure on sample\n#find local minima:\ndef theoretic_sample_p(df):\n    new_df = df.copy()\n    #Create a variable for H2 equiv thermal transpiration corrected Barion 2 values\n    ba2 = df["Barion_2 corr"].values\n    minima_idx = scipy.signal.argrelmin(ba2,order=10)\n    #Find the pressure values corresponding to the indexes found\n    minima_pressure = ba2[minima_idx]\n    #Find the elapsed time values\n    elapsed_time_minima = df["Elapsed time"].values[minima_idx]\n    # Vectorize the theta_hobson function to work with NumPy arrays\n    theta_hobson_vec = np.vectorize(theta_hobson)\n    # Perform curve fitting to get the parameters\n    popt, pcov = curve_fit(theta_hobson_vec, minima_pressure, elapsed_time_minima, p0=1)\n\n    # Calculate the fitted elapsed time values using the theta_hobson function\n    fitted_elapsed_time = theta_hobson_vec(minima_pressure, *popt)\n\n    #Create a polynomial fit that goes through these points\n    # Add data to new_df\n    col_loc =

#### Calculating number of molecules injected
$$
N=\frac{\Sigma dp\cdot V_{inj}}{k_B\cdot T}=\frac{\Sigma (p_{inj,t(x)}-p_{inj,t(x-1)})\cdot V_{inj}}{k_B \cdot T}   \space \left[{M}\right]
$$

$$
N=\frac{\Sigma\frac{dp}{dt}\cdot V_{inj}}{k_B\cdot T}=\frac{\Sigma\frac{(p_{inj,t(x)}-p_{inj,t(x-1)})}{(t_{t(x)}-t_{t(x-1)})}\cdot V_{inj}}{k_B \cdot T}   \space \left[\frac{M}{s}\right]
$$
this formula also takes into account the sampling rate (7s)

In [12]:
 #constants 
V_inj = 6.515e-5
S_sample= 276

""" #Getting the number of molecules injected per second N and (homogenous) coverage:
def N_inj(df, inj_gauge):
    #Calculate dp
    dp = np.diff(df[inj_gauge]*100).clip(0) #mbar to Pa conversion
    # Append a 0 to the beginning of the dif array
    dp = np.insert(dp, 0, 0)
    N = (dp.cumsum()*V_inj)/(scipy.constants.Boltzmann*T1)
    cov = N/S_sample #Molecules per cm2
    col_loc = int(df.columns.get_loc(inj_gauge) + 1)
    df.insert(col_loc,"Number of molecules injected",N)
    df.insert(col_loc+1,"Coverage",cov)
    return df  """

' #Getting the number of molecules injected per second N and (homogenous) coverage:\ndef N_inj(df, inj_gauge):\n   #Calculate dp\n   dp = np.diff(df[inj_gauge]*100).clip(0) #mbar to Pa conversion\n   # Append a 0 to the beginning of the dif array\n   dp = np.insert(dp, 0, 0)\n   N = (dp.cumsum()*V_inj)/(scipy.constants.Boltzmann*T1)\n   cov = N/S_sample #Molecules per cm2\n   col_loc = int(df.columns.get_loc(inj_gauge) + 1)\n   df.insert(col_loc,"Number of molecules injected",N)\n   df.insert(col_loc+1,"Coverage",cov)\n   return df  '

#### Comments for annotations

In [13]:
def comments(dataframe,timecol,commentcol):
    #print hv comments
    #print(pd.unique(dataframe[colname]))
    new_dataframe = dataframe.loc[:,[timecol,commentcol]]
  
    #drop NaN values
    new_dataframe.dropna(inplace=True)
    return new_dataframe

#### Simpson's rule for integration (only for isotherm + ESD mixed measurements)

## Fitting (TODO)

### Adsorption isotherms (DRK, Hobson)



Model describing the physisorption isotherms for low temperatures in the submonolayer region:

$\ln\Theta=-D\left(k_BT\ln\left(\frac{P}{P_0}\right)\right)^2=-B\left(R\,T\ln\left(\frac{P}{P_0}\right)\right)^2$, 

with $k_B$ the Boltzmann constant, $R$ the gas constant, $D$ or $B$ an empirical constant linked to the vaporisation heat of the adsorbate (DR energy)  and $P_0$ the saturated vapor pressure at the temperature $T$.

#### Hobson, 1995
Hobson took $B$ to be $B=3.59\cdot 10^{-6}\,\mathrm{cal^{-2}mol^{2}}$ following Halama and Aggus (https://doi.org/10.1116/1.568583 , 1975).
R = gas_constant=1.9865


#### Chill, Wilfert and Bozyk, 2019
Chill, Wilfert and Bozyk measured isotherms between 7 and 18 K on stainless steel and found by fitting $D=3075\,\mathrm{eV}^{-2}$.

### Calculate pressure on the sample
$$
Q=C\cdot (p_2-p_1(\approx \, <0.1p_2)) \implies p_1 = \frac{Cp_2-Q}{C}
$$
where $p_1$ is the pressure in the RT part and $p_2$ is the pressure in the cold part. C is the conductance of the $d=35$ mm copper chimney
For N2 and $L/R>>1$:
$$
C=12.3\frac{D^3}{L} \left[\frac{l}{s}\right] \implies 12.3\frac{D^3}{L} \cdot \sqrt{\frac{M_{N2}}{M_{H2}}}\left[\frac{l}{s}\right] = 12.3\cdot \frac{3.5^3}{30}\cdot \sqrt{\frac{28}{2}} = 65.77 \frac{l}{s}
$$
where D and L are in cm

In [14]:
""" #Calculate the flux
c_chimney = 423.33 #l/s
def sample_pressure(df):
    new_df = df.copy()
    #define df slice between two comments "turbo valve closed" and "checking base pressure"
    first_inj = new_df.loc[]
    #Take the average pressure value of first_inj
    p2_avg = first_inj["Barion_2 corr"]
    #Take the smallest value of a dataframe slice between comments "checking base pressure" and "restarted injection"
    no_inj = new_df.loc[]
    #Calculate the difference of these values
    delta_p = p2_avg-no_inj
    #Calculate flux according to formula
    Q = c_chimney*delta_p
    #Calculate the sample pressure for every datapoint in new_df according to the formula
    p1 = (c_chimney*p_2-Q)/c_chimney
    #add p1 to the dataframe
    col_loc = int(new_df.columns.get_loc("Barion_2 corr") + 1)
    new_df.insert(col_loc, "sample pressure", p1)
    return """

' #Calculate the flux\nc_chimney = 423.33 #l/s\ndef sample_pressure(df):\n    new_df = df.copy()\n    #define df slice between two comments "turbo valve closed" and "checking base pressure"\n    first_inj = new_df.loc[]\n    #Take the average pressure value of first_inj\n    p2_avg = first_inj["Barion_2 corr"]\n    #Take the smallest value of a dataframe slice between comments "checking base pressure" and "restarted injection"\n    no_inj = new_df.loc[]\n    #Calculate the difference of these values\n    delta_p = p2_avg-no_inj\n    #Calculate flux according to formula\n    Q = c_chimney*delta_p\n    #Calculate the sample pressure for every datapoint in new_df according to the formula\n    p1 = (c_chimney*p_2-Q)/c_chimney\n    #add p1 to the dataframe\n    col_loc = int(new_df.columns.get_loc("Barion_2 corr") + 1)\n    new_df.insert(col_loc, "sample pressure", p1)\n    return '

### $N_{inj}$ by flux integral

$$
N_{inj} = \frac{\int Q dt}{k_BT}
$$ 

In [15]:
c_chimney = 65.77  # l/s

def sample_pressure_and_cov(df):
    new_df = df.copy()

    # Define df slices between all injections, first between comments "turbo valve closed" and "checking base pressure"
    idx_start1 = new_df.loc[(new_df["Live comments"] == "turbo valve closed")].index.min()
    idx_end1 = new_df.loc[(new_df["Live comments"] == "checking baseline")].index.min()
    first_inj = new_df.loc[idx_start1:idx_end1]

    # Define df slices between all injections, second between comments "restarted injection" and "baseline check"
    idx_start2 = new_df.loc[(new_df["Live comments"] == "restarted injection")].index.min()
    idx_end2 = new_df.loc[(new_df["Live comments"] == "baseline check")].index.min()
    second_inj = new_df.loc[idx_start2:idx_end2]

    # Define df slices between all injections, third between comments "continuing injection" and "base pressure check"
    idx_start3 = new_df.loc[(new_df["Live comments"] == "continuing injection")].index.min()
    idx_end3 = new_df.loc[(new_df["Live comments"] == "base pressure check")].index.max()
    third_inj = new_df.loc[idx_start3:idx_end3]

    # Define df slices between all injections, starting from timestamp 18:23
    start_time = 26200 # timestamp after last base pressure check
    start_idx = new_df[new_df["Elapsed time"] >= start_time].index[0]
    fourth_inj = new_df.loc[start_idx:]

    # Take the smallest p value of a datafram slice between comments "checking base pressure" and "restarted injection"
    idx_noinj1_start = new_df.loc[(new_df["Live comments"] == "checking baseline")].index.min()
    idx_noinj1_end = new_df.loc[(new_df["Live comments"] == "restarted injection")].index.min()
    no_inj1 = new_df.loc[idx_noinj1_start:idx_noinj1_end]
    
      # Find the index of the value closest to 2e-10 mbar
    min_pressure1_idx = no_inj1["Barion_2 H2 equiv"].sub(2e-10).abs().idxmin()
    
    # Get the actual value at that index
    min_pressure1 = no_inj1.loc[min_pressure1_idx, "Barion_2 H2 equiv"] 

    # Take the smallest p value of a datafram slice between comments "baseline check"" and "continuing injection"
    idx_noinj2_start = new_df.loc[(new_df["Live comments"] == "baseline check")].index.min()
    idx_noinj2_end = new_df.loc[(new_df["Live comments"] == "continuing injection")].index.min()
    no_inj2 = new_df.loc[idx_noinj2_start:idx_noinj2_end]
    
    # Find the index of the value closest to 2.18e-10 mbar
    min_pressure2_idx = no_inj2["Barion_2 H2 equiv"].sub(2.18e-10).abs().idxmin()
    
    # Get the actual value at that index
    min_pressure2 = no_inj2.loc[min_pressure2_idx, "Barion_2 H2 equiv"] 

    # Take the smallest p value of a datafram slice between comments "base pressure check"" and "restarted injection"
    idx_noinj3_start = new_df.loc[(new_df["Live comments"] == "base pressure check")].index.min()
    idx_noinj3_end = new_df.loc[(new_df["Live comments"] == "restarted injection")].index.max()
    no_inj3 = new_df.loc[idx_noinj3_start:idx_noinj3_end]
    
    # Find the index of the value closest to 2.18e-10 mbar
    min_pressure3_idx = no_inj3["Barion_2 H2 equiv"].sub(2.28e-10).abs().idxmin()
    
    # Get the actual value at that index
    min_pressure3 = no_inj3.loc[min_pressure3_idx, "Barion_2 H2 equiv"] 
    
    # Calculate the flux during third injection
    # Take the average H2 equiv value of first_inj
    p3_avg = (third_inj["Barion_2 H2 equiv"]).mean()

    # Get the non thermal transpiration corrected min_pressure 3
    p3_min = no_inj3.loc[min_pressure3_idx, "Barion_2 H2 equiv"] 

    # Calculate the difference of these values assuming 1) p1 from formula is negligible, and 2) p1 not negligible
    #delta_p = p3_avg 
    delta_p = p3_avg - p3_min

    # Calculate flux according to assumption that 
    const_flux = c_chimney * delta_p

    # Calculate the sample pressure from fourth injection
    p4 = fourth_inj["Barion_2 H2 equiv"][:-1]- (const_flux/c_chimney) #mbarl/s
    
    # Calculate the integrals
    a1 = const_flux*np.diff(first_inj["Elapsed time"])*0.1 #Pa*m3
    a2 = const_flux*np.diff(second_inj["Elapsed time"])*0.1
    a3 = const_flux*np.diff(third_inj["Elapsed time"])*0.1
    a4 = const_flux*np.diff(fourth_inj["Elapsed time"])*0.1

    # Calculate the cumulative sums of Q*dt
    first_interval = np.sum(a1) / (scipy.constants.k * T1) / 276  # molecules/cm2
    second_interval = first_interval + np.sum(a2) / (scipy.constants.k * T1) / 276
    third_interval = second_interval + np.sum(a3) / (scipy.constants.k * T1) / 276
    fourth_interval = third_interval + np.cumsum(a4) / (scipy.constants.k * T1) / 276

    # Add 
    # Define base pressure as the minimum pressure value in first_inj
    p_base = first_inj["Barion_2 H2 equiv"].values.min()

    # Define the corresponding coverage as 0
    cov_base = 0
    
    # Assign a min_pressure value to each interval variable 
    p_arr = np.array([p_base,min_pressure1, min_pressure2, min_pressure3])
    p_arr = np.append(p_arr, p4)
    cov_arr = np.array([cov_base, first_interval, second_interval, third_interval])
    cov_arr = np.append(cov_arr, fourth_interval)

    # Create a new dataframe with columns "pressure" and "coverage". The first four "pressure" datapoints should be base_p, min_pressure1-3 and then p4 values.
    cov = pd.DataFrame({"sample pressure":p_arr, "coverage": cov_arr})

    return new_df, cov


In [16]:
vaclog_comments = comments(vaclog,"Time", "Live comments")
print(vaclog_comments)

NameError: name 'vaclog' is not defined

## Calling the data analysis functions and writing to file

#### Defining variables to the called functions

In [ ]:
# Defining a variable for original measurement data
vaclog = data[".\\vaclog"]
%xl_set vaclog --cell Sheet1!A2

#LHe data
y_val = data[".\\vaclog"]["helium"].values
x_val = data[".\\vaclog"]["Elapsed time"].values
b_full = 560

# Find the indices where x_val has non-NaN values
valid_indices = ~np.isnan(x_val)

# Remove NaN values from both x_val and y_val using the valid_indices
x_val = x_val[valid_indices]
y_val = y_val[valid_indices]

#Straight line fit for LHe data
x_fit, y_fit, a, b = lhe_fit(y_val, x_val)

#Extended fit for LHe data
x_fit_extended, y_fit_extended, a_ext, b_full = ext_fit(y_val, x_val, b_full)

#Cut values before injection start
def rows_from_started_comment(dataframe, start_comment):
    # Find the index where the comment "started injection" is first found
    start_index = dataframe.loc[dataframe["Live comments"] == start_comment].index.min()

    # Extract all rows from the start_index to the end of the DataFrame
    extracted_rows = dataframe.loc[start_index:]

    return extracted_rows

#Calculate Number of molecules and coverage, CernOx temperature, apply gauge correction
vaclog_new = N_inj(rows_from_started_comment((gauge_correction(tempconvert(vaclog))),"turbo valve closed"),"injection 100mbar")
vaclog_sample, coverage = sample_pressure_and_cov(vaclog_new)
#Comments
vaclog_comments = comments(vaclog,"Time", "Live comments")
print(vaclog_comments)
coverage.head()

#print(sample_p(vaclog_new).head())

In [ ]:
print(vaclog_new.head())

In [ ]:
#### Writing to excel file
def write_to_excel(filename):
    #Insert Lhe fit data
    lhe_fit_data = {f"Fit x a={a:.4f}, b={b:.4f}":x_fit,f"Fit y a={a:.4f}, b={b:.4f}":y_fit}
    lhe_ext_fit_data = {f"Extended fit x a={a_ext:.4f}, b={b_full:.4f}":x_fit_extended,f"Extended fit y a={a_ext:.4f}, b={b_full:.4f}":y_fit_extended}
    lhe_fit_df = pd.DataFrame(lhe_fit_data)
    lhe_fit_ext_df = pd.DataFrame(lhe_ext_fit_data)
    
    #Write to excel
    writer = pd.ExcelWriter(os.path.join(os.getcwd(),filename),engine="xlsxwriter")
    vaclog.to_excel(writer,sheet_name='Original data')
    vaclog_new.to_excel(writer, sheet_name='Analysed data')
    lhe_fit_df.to_excel(writer,sheet_name="LHe fit data")
    lhe_fit_ext_df.to_excel(writer,sheet_name="LHe extended fit data")
    writer.save()
    return



## Plotting 

In [ ]:
#define parameters for plotting
#annotation params
font = dict(size = "x-small", color ="green", style ="italic",rotation="vertical")

plt.rcParams["figure.figsize"] = [8,5]
plt.rcParams["axes.edgecolor"] = "black"
plt.rcParams["axes.grid"] = True
plt.rcParams["grid.color"] = "black"
plt.rcParams["grid.linewidth"] = 0.35

formatter = md.DateFormatter('%H:%M')

# Create the "graphs" folder path
graphs_folder = os.path.join(root, 'graphs')

# Ensure the "graphs" folder exists, if not create it
if not os.path.exists(graphs_folder):
    os.makedirs(graphs_folder)
    

In [ ]:
# Plotting the temp evolution
plt.figure()
plt.plot(vaclog_new["Time"],vaclog_new["CernOx Temp"],marker=".", markersize=5,label='T-CERNOX')          
plt.xlabel('Timestamp')
plt.ylabel('Temperature (K)')
plt.legend()
plt.title("Temperature evolution")
plt.yscale('linear')

ax = plt.gca()
ax.xaxis.set_major_formatter(formatter)
ax.xaxis.set_major_locator(plt.MaxNLocator(12))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')    

plt.savefig(os.path.join(graphs_folder,"S2 Temp.png"),dpi=300,bbox_inches='tight')
plt.show()

In [ ]:
# Helium level
fig, ax = plt.subplots()
ax.plot(x_val/3600, y_val, marker=".", markersize=5, label='inj volume pressure')
ax.plot(x_fit/3600, y_fit, "--", color="red", label=f"Fitted line (mm/s): y = {a:.4f}*x+{b:.2f}")
ax.legend(loc="upper right")
ax.set_xlabel('Elapsed time (h)')
ax.set_ylabel('LHe level (mm)')
ax.set_yscale('linear')

plt.title("Helium level drop and fit")
plt.savefig(os.path.join(graphs_folder, 'S2 He_level.png'), dpi=300,bbox_inches='tight')
plt.show()
    

In [ ]:
#Plotting BA2, BA1 pressure w comments as seen
plt.figure(figsize=(12,6))
plt.plot(data[".\\vaclog"]["Time"],data[".\\vaclog"]["Barion_2"],marker=".", markersize=4,label='Barion 2')
plt.plot(data[".\\vaclog"]["Time"],data[".\\vaclog"]["Barion_1"],marker=".", markersize=4,label='Barion 1')
plt.plot(data[".\\vaclog"]["Time"],data[".\\vaclog"]["DUAL experiment"],marker=".", markersize=4,label='DUAL experiment')

plt.xlabel('Timestamp')
plt.ylabel('Pressure (mbar)')
plt.legend(loc="lower right")
plt.title("Ads isotherm gauge readings as read")
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(formatter)
ax.xaxis.set_major_locator(plt.MaxNLocator(12))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')    

#Writing comments as plot annotations
trans  = transforms.blended_transform_factory(
    ax.transData, ax.transAxes)
for i in vaclog_comments.index:
    plt.text(vaclog_comments["Time"][i],0.8,str(vaclog_comments["Live comments"][i]),fontdict=font, transform=trans)      

plt.savefig(os.path.join(graphs_folder, 'S2 gauge readings vs time_N2 equiv.png'), dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#Plotting the non thermal transpiration corrected H2 equiv. gauge readings against elapsed time
plt.figure(figsize=(12,6))
plt.plot(vaclog_new["Elapsed time"],vaclog_new["Barion_1 H2 equiv"],marker=".", c="g",markersize=4,label='Barion 1 H2 equiv')
plt.plot(vaclog_new["Elapsed time"],vaclog_new["Barion_2 H2 equiv"],marker=".", c="y",markersize=4,label='Barion 2 H2 equiv')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Pressure (mbar)')
plt.legend(loc="lower right")
plt.title("Ads isotherm gauge readings H2 equiv vs elapsed time")
plt.yscale('log')

plt.savefig(os.path.join(graphs_folder, 'S2 barion pressure vs elapsed time_h2 equiv.png'), dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#Plotting the thermal transpiration corrected H2 equiv. gauge readings
plt.figure(figsize=(12,6))
plt.plot(vaclog_new["Time"],vaclog_new["Barion_1 H2+tt"],marker=".", c="m",markersize=4,label='Barion 1')
plt.plot(vaclog_new["Time"],vaclog_new["Barion_2 H2+tt"],marker=".", c="c",markersize=4,label='Barion 2')

plt.xlabel('Timestamp')
plt.ylabel('Pressure (mbar)')
plt.legend(loc="lower right")
plt.title("Ads isotherm gauge readings H2 equiv thermal transp corrected")
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(formatter)
ax.xaxis.set_major_locator(plt.MaxNLocator(12))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')    

#Writing comments as plot annotations
trans  = transforms.blended_transform_factory(
    ax.transData, ax.transAxes)
for i in vaclog_comments.index:
    plt.text(vaclog_comments["Time"][i],0.8,str(vaclog_comments["Live comments"][i]),fontdict=font, transform=trans)      

plt.savefig(os.path.join(graphs_folder, 'S2 barion pressure v time. h2 equiv tt corrected pressures.png'), dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#Plotting the real adsorption isotherm (non cov)
fig, ax = plt.subplots()

ax.plot(coverage["coverage"],coverage["sample pressure"],marker=".", markersize=5,label='H2 equiv, no tt corr')
ax.legend(loc="upper left")
ax.set_xlabel('Coverage (M/cm2)')
ax.set_ylabel('Pressure (mbar)')
ax.set_yscale('log')
plt.title("S2 HiLumi sample H2 isotherm. base p + 3 datapoints + cont. isotherm")

plt.savefig(os.path.join(graphs_folder, 'S2 HiLumi sample H2 isotherm. base p + 3 datapoints + cont. isotherm_p_1 not 0_no tt.png'), dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#Plotting the real adsorption isotherm (non cov)
fig, ax = plt.subplots()

ax.plot(coverage["coverage"],coverage["sample pressure"]*tt_coef,marker=".", markersize=5,label='H2 equiv pressure, tt corr')
ax.legend(loc="upper left")
ax.set_xlabel('Coverage (M/cm2)')
ax.set_ylabel('Pressure (mbar)')
ax.set_yscale('log')
plt.title("S2 HiLumi sample H2 isotherm. p_1 =/= 0 in Q=c(p_2-p_1)")

plt.savefig(os.path.join(graphs_folder, 'S2 HiLumi sample H2 isotherm. base p + 3 datapoints + cont. isotherm.p_1 not 0.png'), dpi=300, bbox_inches='tight')
plt.show()

## Execute writing data to file

In [ ]:
#write_to_excel("adsorption isotherms_Tiirinen.xlsx")